In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [10]:
import pandas as pd
def my_analysis(dataset):
    df = pd.DataFrame(columns=['Datatype', 'NA values %', 'Unique values', 'frequent value', 'contribution %', 'min value'])
    length = dataset.shape[0]
    for col in dataset.columns:
        row_df = pd.DataFrame({'Datatype':dataset[col].dtype,
                                'NA values %':round(dataset[col].isna().sum()*100/length,2),
                                'Unique values':dataset[col].nunique(),
                                'frequent value':dataset[col].value_counts().index[0],
                                'contribution %':round(dataset[col].value_counts()[0]*100/length,2),
                                'min value':'none' if dataset[col].dtype=='object' else dataset[col].min()},
                                index=[col])
        df = pd.concat([df,row_df])
    return df

In [12]:
train_data.head()

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,8.0,0.50,0.39,2.2,0.073,30.0,39.0,0.99572,3.33,0.77,12.1,6
1,1,9.3,0.30,0.73,2.3,0.092,30.0,67.0,0.99854,3.32,0.67,12.8,6
2,2,7.1,0.51,0.03,2.1,0.059,3.0,12.0,0.99660,3.52,0.73,11.3,7
3,3,8.1,0.87,0.22,2.6,0.084,11.0,65.0,0.99730,3.20,0.53,9.8,5
4,4,8.5,0.36,0.30,2.3,0.079,10.0,45.0,0.99444,3.20,1.36,9.5,6


In [14]:
train_data.shape

(2056, 13)

In [18]:
train_data['quality'].value_counts()

5    839
6    778
7    333
4     55
8     39
3     12
Name: quality, dtype: int64

In [19]:
y = train_data['quality']-3
x = train_data.drop(['quality','Id'], axis=1)
x_test = test_data.drop('Id', axis=1)

In [20]:
y.value_counts()

2    839
3    778
4    333
1     55
5     39
0     12
Name: quality, dtype: int64

In [25]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, random_state=42) 

In [26]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=500, early_stopping_rounds=5)
xgb.fit(x_train,y_train,eval_set=[(x_val,y_val)])

[0]	validation_0-mlogloss:1.52871
[1]	validation_0-mlogloss:1.38163
[2]	validation_0-mlogloss:1.29191
[3]	validation_0-mlogloss:1.23176
[4]	validation_0-mlogloss:1.19677
[5]	validation_0-mlogloss:1.16592
[6]	validation_0-mlogloss:1.13852
[7]	validation_0-mlogloss:1.12360
[8]	validation_0-mlogloss:1.10596
[9]	validation_0-mlogloss:1.09982
[10]	validation_0-mlogloss:1.09167
[11]	validation_0-mlogloss:1.08627
[12]	validation_0-mlogloss:1.08185
[13]	validation_0-mlogloss:1.07392
[14]	validation_0-mlogloss:1.07177
[15]	validation_0-mlogloss:1.07354
[16]	validation_0-mlogloss:1.07170
[17]	validation_0-mlogloss:1.07554
[18]	validation_0-mlogloss:1.07764
[19]	validation_0-mlogloss:1.07395
[20]	validation_0-mlogloss:1.07209
[21]	validation_0-mlogloss:1.07380


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=5,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [27]:
result = xgb.predict(x_test)

result = [x+3 for x in result]

df = pd.DataFrame({'Id':test_data['Id'].values, 'quality':result})
df.to_csv('submission.csv', index=False)